In [1]:
#Bibliotecas requeridas
import cv2
import cv2 as cv
import glob
import re
import math
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib import colors
from matplotlib.colors import hsv_to_rgb
import os
import numpy as np
import pandas as pd  # Lo agregó Isaac para tener una visualización inmediata de los datos
pd.set_option('display.max_rows', None)
#-------------------------Definición de funciones---------------------------------
#La función switch retorna el número de robot de una combinación de círculo
#grande y círculo pequeño
def switch(numColor1, numColor2):
    if numColor1==1:
        if numColor2==2:
            robot=1 #cyan-amarillo
        elif numColor2==5:
            robot=3 #cyan-verde
        else:
            #caso no identificado con combinaciones actuales
            robot=999
            
    elif numColor1==2:
        if numColor2==1:
            robot=2 #amarillo-cyan
        elif numColor2==5:
            robot=15 #amarillo-verde
        else:
            #caso no identificado con combinaciones actuales
            robot=999
    
    elif numColor1==5:
        if numColor2==1:
            robot=4  #verde-cyan
        elif numColor2==2:
            robot=6  #verde-rosado
        else:
            #caso no identificado con combinaciones actuales
            robot=999
    else:
        #caso no identificado
        robot=999
    return robot
    
def get_contour_areas(cont):
    #Crea una lista donde se guardarán las áreas de cada contorno
    all_areas= []
    #Recorre la lista de contornos
    for cnt in cont:
        #calcula el área de un contorno
        area= cv2.contourArea(cnt)
        #agrega el área calculada al arreglo de áreas
        all_areas.append(area)
    return all_areas

def depurar_contornos(cnts):
    #en esta lista se almacenarán los contornos pertenecientes a los círculos
    #la "d" significa depurados.
    cntsd= []
    #recorro todos los contornos encontrados
    for cnt in cnts:
        area= cv2.contourArea(cnt)
        """
        if area>=500:
            print("Area Encontrada", area)"""
        #Se encontró que el área mínima de los círculos pequeños es mayor a 600 unidades aprox
        #Se encontró que el área  de los círculos grandes es mayor a 1200 unidades aprox
        #cuadradas, daremos un margeddn de error. El limite de 2500 se puso porque en rosado encontraba
        #un contorno no deseado
        if area>=300 and area<6300:
        # Mayor al mínimo de los pequeños y menor al máximo de los grandes y un poquito más
            cntsd.append(cnt)    
    return cntsd

def centros_contornos(cnts, areas):
    centros= []
    #contador
    cont=0
    for s in cnts:
        #calcula los momentos
        M = cv2.moments(s)
        #Calcula el centroide
        #https://docs.opencv.org/3.4/dd/d49/tutorial_py_contour_features.html
        cx = int(M['m10']/M['m00'])
        cy = int(M['m01']/M['m00'])
        #verifica si el contorno es de un círculo pequeño o grande:
        if areas[cont]<=2000 and areas[cont]>=300:
        # Identifica si es un círculo pequeño, límite de cuando son grandes
            #circulo pequeño, se identificará con un 0
            circ=0
        else:
            #circulo grande, se identificará con un 1
            circ=1
                
        centros.append((cx, cy, circ))
        cont+=1
    return centros

def calcular_angulos(xg,xp,yg,yp):    
    #Calculo de cita respecto a la vertical, tomando ángulos positivos en sentido horario
    
    #desplazamiento vertical:
    if xg==xp:        
        if yg>yp:
            cita=180        
        else:
            cita=0    
    else:
        dx=xp-xg        
        dy=yp-yg
        #Cálculo del ángulo en radianes usando arctan2
        cita_rad=np.arctan2(dy,dx)        
        #Conversion a grados        
        cita= np.degrees(cita_rad)
        #Ajuste de ángulo para que esté en rango deseado
        if xp < xg: #Primer y cuarto cuadrante  
            if yp < yg: #  Cuarto cuadrante
                cita += 270
            else:  # Primer cuadrante
                cita -= 90
        elif xp > xg:
            if yp < yg:                
                # Caso xp > xg y yp < yg, se agrega un ajuste de +270 grados (tercer cuadrante)
                cita += 270            
            else:
                # Caso xp > xg y yp >= yg, se agrega un ajuste de -90 grados (segundo cuadrante)                
                cita -= 90
        # Ajuste adicional para asegurarse de que el ángulo esté en el rango 
        if cita > 180:            
            cita -= 360
        elif cita < -180:
            cita += 360
        
    return round(cita,2)

def desplazamiento(centerG, centerP, centrosCG, centrosCP, robot, imagen):
    #centerG e Im2 son las posiciones den las listas de centros
    desplazamientos=[]
    #extracción de las coordenadas de los centros en ambas imágenes (anterior y posterior)
    #print("Centro Imagen 1: ", centerG)
    #center da la posición del centro en la imagen. El 0 indica coordena x, y el 1, y.
    xg=centrosCG[imagen][centerG][0]
    xp=centrosCP[imagen][centerP][0]
    yg=centrosCG[imagen][centerG][1]
    yp=centrosCP[imagen][centerP][1]
    angulo=calcular_angulos(xg,xp,yg,yp)
    #retorna el número de imagen, el robot y cuánto se movió
    out=[imagen, robot, xg - X_origen, yg - Y_origen, angulo]
    #out=[imagen, robot, xg - X_origen, yg - Y_origen, xp - X_origen, yp - Y_origen, angulo]
    desplazamientos.extend(out)
    return desplazamientos

def calcular_desplazamiento(centrosC, centrosR, centrosV, numImagenes, lista_robots):
    #resolución espacial para imagen [mm/pixel]
    mmPixel=40/23
    #crea la lista donde se almacenarán los desplazamientos entre 2 imágenes del
    #mismo identificador.
    #n contiene el número de imágenes
    desplazamientos=[]
    #lista de números de robots
    Robots=[]
    
    #analiza todas las imágenes en pares: momento anterior y momento posterior, hasta n-1 imágenes
    
    #cada peers contendrá el número de imagen, los colores pareja,
    #el centro x,y de cada contorno que es pareja. Peers devuelve
    #desde 0 parejas, hasta el número de combinaciones de un color de
    #círculo grande con el resto de colores.
    
    if (1 in lista_Robots):
        #Cyan-Rosado
        CApeers=buscar_peer(1, 2, centrosC, centrosR)
        #calcula desplazamientos solo si encontró robots
        if len(CApeers)!=0:
            #se asume el robot NO desaparece de la escena
            #k es el número de imagen
            for k in range (numImagenes-1):
                #pareja es el número de pareja en la k-ésima imagen
                for pareja in range(len(CApeers[k])):
                    #la última posición que es 2 indica que extrae el centro del círculo grande
                    centerG=CApeers[k][pareja][2]
                    #extrae el centro pequeño de la pareja
                    centerP=CApeers[k][pareja][3]
                    #la posición 1 indica el número de robot al que corresponde la pareja
                    robot= CApeers[k][pareja][1]
                    #movimiento devuelve el #imagen, el robot y cuánto se desplazó
                    movimiento=desplazamiento(centerG, centerP, centrosC, centrosR, robot, k)
                    desplazamientos.append(movimiento)
    
    if (2 in lista_Robots):
        #Rosado-cyan
        ACpeers=buscar_peer(2, 1, centrosR, centrosC)
        #calcula desplazamientos solo si encontró robots
        if len(ACpeers)!=0:
            #se asume el robot NO desaparece de la escena
            #k es el número de imagen
            for k in range (numImagenes-1):
                #pareja es el número de pareja en la k-ésima imagen
                for pareja in range(len(ACpeers[k])):
                    #la última posición que es 2 indica que extrae el centro del círculo grande
                    centerG=ACpeers[k][pareja][2]
                    #extrae el centro grande de la pareja pero en la imagen siguiente
                    centerP=ACpeers[k][pareja][3]
                    #la posición 1 indica el número de robot al que corresponde la pareja
                    robot= ACpeers[k][pareja][1]
                    #movimiento devuelve el #imagen, el robot y cuánto se desplazó
                    movimiento=desplazamiento(centerG,centerP, centrosR, centrosC, robot, k)
                    desplazamientos.append(movimiento)
                        
    if (3 in lista_Robots):
        #Cyan-verde
        CVpeers=buscar_peer(1, 5, centrosC, centrosV)
        #calcula desplazamientos solo si encontró robots
        if len(CVpeers)!=0:
            #se asume el robot NO desaparece de la escena
            #k es el número de imagen
            for k in range (numImagenes-1):
                #pareja es el número de pareja en la k-ésima imagen
                for pareja in range(len(CVpeers[k])):
                    #la última posición que es 2 indica que extrae el centro del círculo grande
                    centerG=CVpeers[k][pareja][2]
                    #extrae el centro grande de la pareja pero en la imagen siguiente
                    centerP=CVpeers[k][pareja][3]
                    #la posición 1 indica el número de robot al que corresponde la pareja
                    robot= CVpeers[k][pareja][1]
                    #movimiento devuelve el #imagen, el robot y cuánto se desplazó
                    movimiento=desplazamiento(centerG,centerP, centrosC, centrosV, robot, k)
                    desplazamientos.append(movimiento)
    
    if (4 in lista_Robots):
        #Verde-Cyan
        VCpeers=buscar_peer(5, 1, centrosV, centrosC)
        #calcula desplazamientos solo si encontró robots
        if len(VCpeers)!=0:
            #se asume el robot NO desaparece de la escena
            #k es el número de imagen
            for k in range (numImagenes-1):
                #pareja es el número de pareja en la k-ésima imagen
                for pareja in range(len(VCpeers[k])):    #CVpeers
                    #la última posición que es 2 indica que extrae el centro del círculo grande
                    centerG=VCpeers[k][pareja][2]
                    #extrae el centro grande de la pareja pero en la imagen siguiente
                    centerP=VCpeers[k][pareja][3]
                    #la posición 1 indica el número de robot al que corresponde la pareja
                    robot= VCpeers[k][pareja][1]
                    #movimiento devuelve el #imagen, el robot y cuánto se desplazó
                    movimiento=desplazamiento(centerG,centerP, centrosV, centrosC, robot, k)
                    desplazamientos.append(movimiento)

    if (5 in lista_Robots):
        #Rosado-Verde    
        MVpeers=buscar_peer(2, 5, centrosR, centrosV)
        #calcula desplazamientos solo si encontró robots
        if len(MVpeers)!=0:
            #se asume el robot NO desaparece de la escena
            #k es el número de imagen
            for k in range (numImagenes-1):
                #pareja es el número de pareja en la k-ésima imagen
                for pareja in range(len(MVpeers[k])):
                    #la última posición que es 2 indica que extrae el centro del círculo grande
                    centerG=MVpeers[k][pareja][3]
                    #extrae el centro grande de la pareja pero en la imagen siguiente
                    centerP=MVpeers[k][pareja][2]
                    #la posición 1 indica el número de robot al que corresponde la pareja
                    robot= MVpeers[k][pareja][1]
                    #movimiento devuelve el #imagen, el robot y cuánto se desplazó
                    movimiento=desplazamiento(centerG,centerP, cetntrosR, centrosV, robot, k)
                    desplazamientos.append(movimiento)
                        
    if (6 in lista_Robots):
        #Verde-rosado   
        VMpeers=buscar_peer(5, 2, centrosV, centrosR)
        #print(VMpeers)
        #calcula desplazamientos solo si encontró robots
        if len(VMpeers)!=0:
            #se asume el robot NO desaparece de la escena
            #k es el número de imagen
            for k in range (numImagenes-1):
                #pareja es el número de pareja en la k-ésima imagen
                for pareja in range(len(VMpeers[k])):
                    #la última posición que es 2 indica que extrae el centro del círculo grande
                    centerG=VMpeers[k][pareja][2]
                    #extrae el centro grande de la pareja pero en la imagen siguiente
                    centerP=VMpeers[k][pareja][3]
                    #la posición 1 indica el número de robot al que corresponde la pareja
                    robot= VMpeers[k][pareja][1]
                    #movimiento devuelve el #imagen, el robot y cuánto se desplazó
                    movimiento=desplazamiento(centerG,centerP, centrosV, centrosR, robot, k)
                    desplazamientos.append(movimiento)
                        
    if (7 in lista_Robots):
        #Rosado-cyan
        RCpeers=buscar_peer(2, 1, centrosR, centrosC)
        #calcula desplazamientos solo si encontró robots
        if len(RCpeers)!=0:
            #se asume el robot NO desaparece de la escena
            #k es el número de imagen
            for k in range (numImagenes-1):
                #pareja es el número de pareja en la k-ésima imagen
                for pareja in range(len(RCpeers[k])):
                    #la última posición que es 2 indica que extrae el centro del círculo grande
                    centerG=RCpeers[k][pareja][2]
                    #extrae el centro grande de la pareja pero en la imagen siguiente
                    centerP=RCpeers[k][pareja][3]
                    #la posición 1 indica el número de robot al que corresponde la pareja
                    robot= RCpeers[k][pareja][1]
                    #movimiento devuelve el #imagen, el robot y cuánto se desplazó
                    movimiento=desplazamiento(centerG,centerP, centrosR, centrosC, robot, k)
                    desplazamientos.append(movimiento)

    if (8 in lista_Robots):
        #Cyan-rosado
        CRppers=buscar_peer(1, 2, centrosC, centrosR)
        #calcula desplazamientos solo si encontró robots
        if len(CRppers)!=0:
            #se asume el robot NO desaparece de la escena
            #k es el número de imagen
            for k in range (numImagenes-1):
                #pareja es el número de pareja en la k-ésima imagen
                for pareja in range(len(CRppers[k])):
                    #la última posición que es 2 indica que extrae el centro del círculo grande
                    centerG=CRppers[k][pareja][2]
                    #extrae el centro grande de la pareja pero en la imagen siguiente
                    centerP=CRppers[k][pareja][3]
                    #la posición 1 indica el número de robot al que corresponde la pareja
                    robot= CRppers[k][pareja][1]
                    #movimiento devuelve el #imagen, el robot y cuánto se desplazó
                    movimiento=desplazamiento(centerG,centerP, centrosC, centrosR, robot, k)
                    desplazamientos.append(movimiento)
                        
    return desplazamientos

def escribir_txt(nombretxt,desplazamientos):
    with open(nombretxt, 'w') as output:
        output.write("99 = es una posición inicial; robot; coordenada x en mm; coordenada y en mm; ángulo respecto a la horizontal"+ '\n')
        output.write("\nDesplazamientos: #imagen; robot; x (mm); y (mm); ángulo (°)"+ '\n')
        for d in desplazamientos:
            imagen=d[0]
            robot=d[1]
            x=d[2]
            y=d[3]
            angulo=d[4]
            output.write(str(imagen) + ";"+str(robot)+ ";"+ str(x)+";"+ str(y)+";"+ str(angulo)+ '\n')
        print('\n')
        print("Archivo creado")
        
def buscar_peer(numColor1, numColor2, centrosC1, centrosC2):
    #la lista de peers contendrá los colores que son pareja (su identificador), y la posición
    #de cada círculo en la lista de centros.
    peers=[]
    for imagen in range(len(centrosC1)):
        #almacenará todos los peers de una imagen para los 2 colores dados
        peersXimagen=[]
        for centro in range(len(centrosC1[imagen])):
            for centro2 in range(len(centrosC2[imagen])):
                #extracción de las coordenadas de los centros de ambos colores
                x1=centrosC1[imagen][centro][0]
                x2=centrosC2[imagen][centro2][0]
                y1=centrosC1[imagen][centro][1]
                y2=centrosC2[imagen][centro2][1]
                #print("x1,x2,y1,y2", x1, x2, y1,y2)
                distance = math.sqrt( ((x1-x2)**2)+((y1-y2)**2) )
                #55 píxeles mide de diámetro aproximadamente un círculo grande
                #se aproxima la mitad de esta distancia como una distancia prudencial
                #para encontrar un contorno de círculo pequeño. Además, la distancia aproximada entre centros
                #de círculos grande y pequeño es de 30 (comprobado por paint y este cálculo). Se agregará un pequeño margen de 5 píxeles
                #Esta función solo agrega al peer si la distancia se cumple y el primero es el más grande, así
                #se evita la duplicidad de peers.
                if (distance<=35) and (centrosC2[imagen][centro2][2]==0):
                    #guarda el número de imagen en cuestión, los colores que hicieron match,
                    #las posiciones dentro de los centros de cada color que hicieron match.
                    #identificar #robot:
                    robot=switch(numColor1, numColor2)
                    peer=(imagen, robot, centro, centro2)
                    """if centro2==0:
                        print("peer encontrado", peer)"""
                    #agrega cada peer encontrado a la lista de peers
                    peersXimagen.append(peer)
                    
        peers.append(peersXimagen)
    #al final peers tendrá un número de elementos igual al del número de imágenes
    #cada elemento es una lista de parejas encontradas para los 2 colores solicitados
    return peers

def robot_init(lista_robots, centrosC, centrosA, centrosV):
    #determina las coordenadas iniciales del robot que se encuentra al inicio de la fila india (más a la izquierda)
    #busca el robot con la coordenada X más pequeña
    
    xInicialMenor = 10000 #se coloca un número grande para iniciar el ordenamiento de menor a mayor
    yInicial = 0
    #Verifica si hay robots con cyan 
    if len(centrosC[0])!=0:
        for k in centrosC[0]:
            if k[2]==1 and k[0]<xInicialMenor:  #círculo grande
                xInicialMenor = k[0]
                yInicial = k[1]
                
    #Verifica si hay robots con amarillo
    if len(centrosA[0])!=0:
        for k in centrosA[0]:
            if k[2]==1 and k[0]<xInicialMenor:  #círculo grande
                xInicialMenor = k[0]
                yInicial = k[1]
                
    #Verifica si hay robots con verde
    if len(centrosV[0])!=0:
        for k in centrosV[0]:
            if k[2]==1 and k[0]<xInicialMenor:  #círculo grande
                xInicialMenor = k[0]
                yInicial = k[1]
    
    return xInicialMenor, yInicial

#calculo de angulo entre rectas
#https://byjus.com/maths/angle-between-two-lines/

def obtenerFrames(nombreVideoIn, nombreCarpetaOut):
    #Se extraen los cuadros del video y se guardan
    #El vídeo está en 30 fps
    
    cap= cv2.VideoCapture(nombreVideoIn)
    n=0
    contFrames=0
    contImagenes=0
    #cada cuántos frames guardo una imagen
    espaciado=6  #aprox cada 10 segundos tomando en cuenta 30fps  #REBECA
    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret == False:
            break
        #este
        if contFrames==espaciado:
            cv2.imwrite(str(nombreCarpetaOut)+'/frame'+str(contImagenes)+'.jpg',frame)
            contImagenes+=1
            contFrames=0
        n+=1
        contFrames+=1
    cap.release()
    cv2.destroyAllWindows()
    

    #Se cargan los cuadros del video en una colección
    #primero se guardan los nombres de las imagenes
    col_images = []
    col_frames = os.listdir(str(nombreCarpetaOut))

    # se ordenan por nombre, en este caso se guardaron con numeración consecutiva
    col_frames.sort(key=lambda f: int(re.sub('\D', '', f)))
    #returna el número de frames de la imagen
    #con este número y la duración del vídeo se obtienen los fps
    return n
        

#--------------------- Lectura de imágenes-----------------------------------------
#Nombre del video y carpeta para guardar las imágenes correspondientes
nombreVideoIn='2024-02-12_14-06-46.mp4'
nombreCarpetaOut='ImagenesRbk/'

#---------------------------PARA PROBAR OTRO VÍDEO
#Obtener Frames de un vídeo: Solo se ejecuta si es la primera vez con este vídeo
numFrames=obtenerFrames(nombreVideoIn, nombreCarpetaOut)
#-------------------------------------------------


#Dirección de la carpeta con las imágenes a utilizar

#Dirección de imágenes prueba Bryan
#dir = 'Imagenes/'
#Dirección de imágenesde prueba Rebeca
dir = nombreCarpetaOut
#Guarda los nombres de las imágenes que estaban en la carpeta en la lista imgList
imgList = os.listdir(dir)
#Inicializa la lista que contendrá a las imágenes que se utilizarán
imagenes = []
imagenesHSV = []
#Ordena las imágenes por número de menor a mayor
imgList.sort (key = lambda x: int (x.replace ("frame", ""). split ('.') [0])) 
#Inicializa contadores
n = 0
cont = 0
ind = 0
#resolución espacial para imagen [mm/pixel]
mmPixel=40/23

#Carga los matrices de calibración
mtx = np.loadtxt('cameraMatrix.txt', dtype=float)
dist = np.loadtxt('dist.txt', dtype=float)

for i in imgList:
    #va contando los frames, prevista para frames
    #cont+=1
    #Lee cada imagen, donde i es el nombre de cada una
    img_conDist = cv2.imread(nombreCarpetaOut+i)
   
    h,  w = img_conDist.shape[:2]
    newcameramtx, roi = cv.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))
    
    #Elimina distorsión
    dst = cv.undistort(img_conDist, mtx, dist, None, newcameramtx)
    
    # recorta la imagen
    x, y, w, h = roi
    
    #Imagen sin distorsión
    img = dst[y:y+h, x:x+w]            
    
    #Convirtiendo a RGB
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    
    #Guarda las imágenes en RGB en una lista
    imagenes.append(img)
    
    #Guarda las imágenes en HSV en una lista
    imagenesHSV.append(hsv)
    #va contando las imágenes que hay
    n+=1

#--------------------- Definición de rangos de color HSV---------------------------
#Rangos cyan:
light_cyan = (95, 157, 145)
dark_cyan = (105, 255, 250) 

#Rangos rosado:
light_rosado = (155, 200, 180)  # 135, 140, 190
dark_rosado = (185, 255, 255)   # 190, 240, 250

#Rangos verde:  
light_verde = (30, 125, 125)
dark_verde = (40, 255, 255)

#Rangos amarillo: 
light_ama = (12, 25, 175)
dark_ama = (35, 255, 255)

#Crea un respaldo de todas las imágenes para ser modificadas más adelante
RespaldoImagenes=list(imagenes)

#------------Procesado: aplicar máscara, dilatar, contornos, ubicación -----------------
#Aquí se almacenarán los contornos de cada una de las imágenes
lista_cCyan = []
lista_cRosado = []
lista_cVerde = []
lista_cAmarillo = []

#Aquí se almacenarán los centros de los contornos
centros_cCyan = []
centros_cRosado = []
centros_cVerde = []
centros_cAmarillo = []

for j in range(len(imagenes)):
    
    #Pone 1 en el píxel que está dentro del rango, sino pone 0
    
    #Máscara Cyan
    maskcyan = cv2.inRange(imagenesHSV[j], light_cyan, dark_cyan)
    #Máscara rosado
    maskrosado = cv2.inRange(imagenesHSV[j], light_rosado, dark_rosado)
    #Máscara Verde
    maskverde = cv2.inRange(imagenesHSV[j], light_verde, dark_verde)
    #Máscara Amarillo
    maskamarillo = cv2.inRange(imagenesHSV[j], light_ama, dark_ama)
    
    #Máscara de todos los colores de identificadores:
    maskAll=maskcyan+maskverde+maskrosado+maskamarillo

    #Se utiliza una transformación morfológica de dilatación de la imagen para hacer los
    #contornos más uniformes
    #Se crea el kernel, entre mayor sea más "grosera" es la dilatación
    kernel = np.ones((7,7),np.uint8)
    cyand = cv2.dilate(maskcyan,kernel,iterations=1)
    rosadod = cv2.dilate(maskrosado,kernel,iterations=1)
    verded = cv2.dilate(maskverde,kernel,iterations=1)
    amarillod = cv2.dilate(maskamarillo,kernel,iterations=1)
    alld = cyand+rosadod+verded+amarillod
    
    #Aplica las máscaras a las imágenes por si se quisieran visualizar
    #"""
    resultcyan = cv2.bitwise_and(imagenes[j], imagenes[j], mask=cyand)
    resultrosado = cv2.bitwise_and(imagenes[j], imagenes[j], mask=rosadod)
    resultverde = cv2.bitwise_and(imagenes[j], imagenes[j], mask=verded)
    resultamarillo = cv2.bitwise_and(imagenes[j], imagenes[j], mask=amarillod)
    #Aplica la máscara combinada
    resultAll = cv2.bitwise_and(imagenes[j], imagenes[j], mask=maskAll)
    
    #se inicializa la variable de tamaño de contorno
    sizeC=0
    #Busca los contornos en cada una de las imágenes filtradas
    #Se aproximan por medio de una compresión horizontal, vertical y diagonal
    #https://docs.opencv.org/master/d3/dc0/group__imgproc__shape.html#gadf1ad6a0b82947fa1fe3c3d497f260e0
    cntsCyan, jerCyan = cv2.findContours(cyand.copy(),cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    cntsRosado, jerrosado = cv2.findContours(rosadod.copy(),cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    cntsVerde, jerVerde = cv2.findContours(verded.copy(),cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    cntsAmarillo, jerAmarillo = cv2.findContours(amarillod.copy(),cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
      
    #Deja solo los contornos grandes, correspondientes a los círculos pequeños y grandes
    cntsCyans=depurar_contornos(cntsCyan)
    cntsRosados=depurar_contornos(cntsRosado)
    cntsVerdes=depurar_contornos(cntsVerde)
    cntsAmarillos=depurar_contornos(cntsAmarillo)
    
    #Se ordenan los contornos por área, la "s" al final de la nueva variable es de "sorted"
    #sorted ordena de menor a mayor por área los contornos.
    cntsCyans = sorted(cntsCyans, key=lambda x: cv2.contourArea(x))
    cntsRosados = sorted(cntsRosados, key=lambda x: cv2.contourArea(x))
    cntsVerdes = sorted(cntsVerdes, key=lambda x: cv2.contourArea(x))
    cntsAmarillo = sorted(cntsAmarillos, key=lambda x: cv2.contourArea(x))
    
    #Se agregan los contornos encontrados por cada color a la lista de contornos correspondiente
    lista_cCyan.append(cntsCyans)
    lista_cRosado.append(cntsRosados)
    lista_cVerde.append(cntsVerdes)
    lista_cAmarillo.append(cntsAmarillos)
    
    #Calcula el área de los contornos encontrados
    #Se observa que las áreas pequeñas (círculos pequeños), son de al menos 5000 unidades cuadradas,
    #mientras que las áreas de círculos grandes están en el orden de 7000-12000 unidades cuadradas
    #Por este motivo, se proceden a eliminar los contornos que no cumplan estos criterios.
    areasCyan=get_contour_areas(cntsCyans)
    areasRosado=get_contour_areas(cntsRosados)
    areasVerde=get_contour_areas(cntsVerdes)
    areasAmarillo=get_contour_areas(cntsAmarillos)
    
    #Cálculo de los centroides de los contornos
    centrosCyan=centros_contornos(cntsCyans, areasCyan)
    centrosRosado=centros_contornos(cntsRosados, areasRosado)
    centrosVerde=centros_contornos(cntsVerdes, areasVerde)
    centrosAmarillo=centros_contornos(cntsAmarillos, areasAmarillo)
    
    #Agrego los centros de cada color a su respectiva lista que contiene a los
    #centros de todas las imágenes
    #Fueron verificados en paint: validados!
    centros_cCyan.append(centrosCyan)
    centros_cRosado.append(centrosRosado)
    centros_cVerde.append(centrosVerde)
    centros_cAmarillo.append(centrosAmarillo)

#Pregunta al usuario cuales robots se usarán

print("---------LISTA DE ROBOTS DISPONIBLES----------")
print("\n")
print("1. Robot Cyan-Amarillo")
print("2. Robot Amarillo-Cyan")
print("3. Robot Cyan-Verde")
print("4. Robot Verde-Cyan")
print("5. Robot Rosado-Verde")
print("6. Robot Verde-Rosado")
print("7. Robot Rosado-Cyan")
print("8. Robot Cyan-Rosado")
print("\n")
print("Ej: 1 3 4 6")
print("\n")
lista_usuario = input("Indique el número de cada robot que se vaya a usar: ")    
lista_Robots = lista_usuario.split()
for i in range(len(lista_Robots)):
    lista_Robots[i] = int(lista_Robots[i])

    
#Determina cuál es el robot inicial de la fila india
X_origen, Y_origen = robot_init(lista_Robots, centros_cCyan, centros_cRosado, centros_cVerde)
#Cálculo de desplazamientos
desplazamientosX=calcular_desplazamiento(centros_cCyan, centros_cRosado, centros_cVerde, n, lista_Robots)

# ISAAC: Agregué esto para imprimir las coordenadas de los centros de un círculo grande
   
desplazamientosX = np.array(desplazamientosX)

df = pd.DataFrame(desplazamientosX, columns=['#Img', 'ID','xG (mm)', 'yG (mm)', 'Ángulos(°)'])
#df = pd.DataFrame(desplazamientosX, columns=['#Img', 'ID','xG', 'yG', 'xP', 'yP', 'Ángulos(°)'])
print(df)

#print(desplazamientosX)

#Crea archivo txt con los desplazamientos
#Nombre del txt:
#nombretxt="Desplazamientos.txt" #REBECA
nombretxt="Tram_CV_140621.txt"
escribir_txt(nombretxt,desplazamientosX)

---------LISTA DE ROBOTS DISPONIBLES----------


1. Robot Cyan-Amarillo
2. Robot Amarillo-Cyan
3. Robot Cyan-Verde
4. Robot Verde-Cyan
5. Robot Rosado-Verde
6. Robot Verde-Rosado
7. Robot Rosado-Cyan
8. Robot Cyan-Rosado


Ej: 1 3 4 6


Indique el número de cada robot que se vaya a usar: 3 4 5
      #Img   ID  xG (mm)  yG (mm)  Ángulos(°)
0    158.0  3.0  -9196.0    728.0       91.91
1    165.0  3.0  -9135.0    725.0       64.29
2    166.0  3.0  -9132.0    727.0       22.48
3    167.0  3.0  -9129.0    726.0      -13.13
4    168.0  3.0  -9130.0    724.0      -16.19
5    169.0  3.0  -9130.0    724.0      -16.70
6    170.0  3.0  -9130.0    724.0      -16.70
7    171.0  3.0  -9129.0    725.0      -14.93
8    172.0  3.0  -9129.0    725.0      -14.93
9    173.0  3.0  -9129.0    725.0      -14.93
10   174.0  3.0  -9129.0    724.0      -14.04
11   175.0  3.0  -9129.0    724.0      -14.04
12   176.0  3.0  -9129.0    724.0      -15.71
13   177.0  3.0  -9129.0    724.0      -14.47
14   178.0  3.0